In [ ]:
Проведем те же манипуляции с датасетом amazon product review

In [ ]:
Таким образом у нас будут датасеты на английском:
    - 20newsGroups
    - CoLA
    - Amazon Product Reviews
    
На русском:
    - news sentiment analysis in Russian

In [ ]:
Модели, которые мы рассматриваем:
    - лог рег на тф-идф
    - БЕРТ английский, мультиязычный, русский
    
Атаки: 
    - BAE, TextFooler (нужны русские эмбеддинги, чтобы запустить на русском(аналог Glove))

In [ ]:
Уточнение насчет данных:
    2 датасета на язык 
    - общий домен и специфический домен!

## Amazon product review

In [8]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

In [ ]:
ссылка на датасет:
    https://www.kaggle.com/c/amazon-pet-product-reviews-classification

In [10]:
# Загрузим данные

!ls data/amazon/

amazon-pet-product-reviews-classification (3).zip
sample_submission.csv
test.csv
train.csv
unlabeled.csv
valid.csv


In [11]:
pwd

'/Users/alexeyzapolskii/Desktop/Local_adversarial_folder/text_attacks'

In [12]:
# to reproduce, download the data and customize this path
PATH_TO_DATA = 'data/amazon/'

In [13]:
train_df = pd.read_csv(PATH_TO_DATA + 'train.csv', index_col='id').fillna('')
valid_df = pd.read_csv(PATH_TO_DATA + 'valid.csv', index_col='id').fillna('')
test_df = pd.read_csv(PATH_TO_DATA + 'test.csv', index_col='id').fillna('')

In [14]:
train_df.shape, valid_df.shape, test_df.shape

((52057, 2), (17353, 2), (17353, 1))

In [ ]:
52k в трейне, 17к в валидац и 17к в тесте

In [15]:
train_df.head() #только две колонки - текст и лейбл

,text,label
id,,
0,Sam has an everlast treat each nite before bed...,dogs
1,The product is as it says. I keep an eye on it...,dogs
2,My Kitty thinks these are treats! He loves the...,dogs
3,This is the third or fourth time that we've or...,dogs
4,Put this on both my dogs. And they are scratch...,dogs


In [16]:
#Stop words present in the library

from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
#print(stopwords.words('english'))

#library that contains punctuation
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
stopwords = stopwords.words("english")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in stopwords\
              and token != " " \
              and token.strip() not in string.punctuation]
    
    text = " ".join(tokens)
    
    return text

In [22]:
print(train_df.head().text[0])

Sam has an everlast treat each nite before bed, like a good tooth brushing. The only downside is finding a place that keeps them in stock as well as multiple flavors.


In [23]:
print(preprocess_text(train_df.head().text[0]))

sam everlast treat nite bed like good tooth brushing downside finding place keeps stock well multiple flavors


In [24]:
train_df.head()

,text,label
id,,
0,Sam has an everlast treat each nite before bed...,dogs
1,The product is as it says. I keep an eye on it...,dogs
2,My Kitty thinks these are treats! He loves the...,dogs
3,This is the third or fourth time that we've or...,dogs
4,Put this on both my dogs. And they are scratch...,dogs


In [25]:
X = train_df.text.apply(lambda x: preprocess_text(x))
y = train_df.label


In [26]:
X.shape, y.shape

((52057,), (52057,))

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [29]:
vectoriser = TfidfVectorizer()

X_train = vectoriser.fit_transform(X_train)
# Convert sparse matrix to dataframe
#X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

# Transform to feature matrix
X_test = vectoriser.transform(X_test)
# Convert sparse matrix to dataframe
#X_test = pd.DataFrame.sparse.from_spmatrix(X_test)
# Add column names to make it more readible



In [30]:
X_train.shape, X_test.shape

((34878, 34282), (17179, 34282))

In [31]:
lg = LogisticRegression(max_iter=10000)

In [32]:
lg.fit(X_train, y_train) # сошлось

LogisticRegression(max_iter=10000)

In [33]:
# лемматизация

print(classification_report(y_test, lg.predict(X_test))) #качество на тесте

                      precision    recall  f1-score   support

               birds       0.88      0.55      0.68       342
bunny rabbit central       0.69      0.31      0.43       192
                cats       0.82      0.75      0.78      6032
                dogs       0.82      0.91      0.86      9278
   fish aquatic pets       0.97      0.78      0.87      1218
       small animals       0.64      0.18      0.28       117

            accuracy                           0.83     17179
           macro avg       0.80      0.58      0.65     17179
        weighted avg       0.83      0.83      0.82     17179



In [35]:
test_df.head()

,text
id,
0,Great Filter. I have always used Marineland fi...
1,This is a great item. My elderly dog loves it....
2,"I have a shep/akita mix, but I bought this mor..."
3,I'm always amazed by what is available on Amaz...
4,I have 2 Abyssinians who will use any litterbo...


In [36]:
X_holdout = test_df.text.apply(lambda x: preprocess_text(x))


In [37]:
X_holdout = vectoriser.transform(X_holdout)



In [38]:
pred = lg.predict(X_holdout)

In [40]:
pred.shape, X_holdout.shape

((17353,), (17353, 34282))

In [42]:
!ls data/amazon/

amazon-pet-product-reviews-classification (3).zip
sample_submission.csv
test.csv
train.csv
unlabeled.csv
valid.csv


In [43]:
sample_sub_df = pd.read_csv(PATH_TO_DATA + 'sample_submission.csv',
                           index_col='id')

In [46]:
sample_sub_df.head()

,label
id,
0,fish aquatic pets
1,dogs
2,dogs
3,dogs
4,cats


In [45]:
sample_sub_df['label'] = pred


In [47]:
sample_sub_df.to_csv('tf-idf-zapolskii.csv')